In [ ]:
import os
import glob
import pandas as pd
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics"

all_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []

for file_path in all_files:

    df_temp = pd.read_csv(file_path)

    file_name = os.path.splitext(os.path.basename(file_path))[0]

    df_temp["source_file"] = file_name

    df_list.append(df_temp)

df_combined = pd.concat(df_list, ignore_index=True)

output_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data.csv"

df_combined.to_csv(output_path, index=False)
print(f"Combined data saved to: {output_path}")


In [ ]:
print(folder_path)


In [ ]:
import pandas as pd

# Provide the full path to all_data.csv in Google Drive
csv_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data.csv"
df = pd.read_csv(csv_path)

# Filter rows by text length in the 4th column
df_filtered = df[(df.iloc[:, 4] > 10) & (df.iloc[:, 4] <= 250)]

# Save output
output_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv"
df_filtered.to_csv(output_path, index=False)
print(f"Filtered data saved to: {output_path}")


In [ ]:
import pandas as pd
import math

# 1. Read the filtered data
input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv"
df = pd.read_csv(input_path)

# 2. Shuffle the rows
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 3. Split into multiple CSV files, each with up to 200 rows
max_rows = 200
total_rows = len(df_shuffled)

# Calculate how many chunks are needed
num_chunks = math.ceil(total_rows / max_rows)

for i in range(num_chunks):
    # Start and end index for the chunk
    start_index = i * max_rows
    end_index = min(start_index + max_rows, total_rows)

    # Slice the DataFrame
    chunk = df_shuffled.iloc[start_index:end_index]

    # Name the output file (e.g., part_1, part_2, ...)
    output_path = f"/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_{i+1}.csv"
    chunk.to_csv(output_path, index=False)
    print(f"Saved {output_path} with {len(chunk)} rows.")


In [ ]:
import pandas as pd
import requests

input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_1.csv"
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

# Function to send text to DeepSeek and get responses
def analyze_text_with_deepseek(text):
    # Replace with your DeepSeek API endpoint and API key
    api_url = "https://api.deepseek.com/v1/analyze"
    headers = {
        "Authorization": "",
        "Content-Type": "application/json"
    }

    # Combine prompts with the text
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    # Send request to DeepSeek
    response = requests.post(api_url, headers=headers, json={"prompt": full_prompt})

    if response.status_code == 200:
        return response.json()  # Assuming DeepSeek returns a JSON response
    else:
        return {"error": f"API request failed with status code {response.status_code}"}

# Apply the function to each row in the CSV
df['Analysis'] = df.iloc[:, 3].apply(analyze_text_with_deepseek)

# Save the results to a new CSV file
df.to_csv('analyzed_data.csv', index=False)

print("Analysis complete. Results saved to 'analyzed_data.csv'.")

In [ ]:
import pandas as pd
from openai import OpenAI

# Initialize the OpenAI client with DeepSeek's API
client = OpenAI(api_key="", base_url="https://openrouter.ai/api/v1")

input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_2.csv"
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

# Function to send text to DeepSeek and get responses
def analyze_text_with_deepseek(text):
    # Combine prompts with the text
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    # Send request to DeepSeek
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": full_prompt},
        ],
        stream=False
    )

    # Extract the response content
    return response.choices[0].message.content

# Apply the function to the fourth column (index 3) in each row
df['Analysis'] = df.iloc[:, 3].apply(analyze_text_with_deepseek)

# Save the results to a new CSV file
df.to_csv('analyzed_data.csv', index=False)

print("Analysis complete. Results saved to 'analyzed_data.csv'.")

In [1]:
import ollama
from tqdm import tqdm

prompts = ["""Understand the following definitions:
    All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
    Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
    Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
    Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
    Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
    Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
    Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
    Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
    Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
    Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
    What if?: I keep asking myself questions such as “what if something happens?”
    Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
    Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
    Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
    Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.""",
"", # left empty for the text to be analyzed.
"What is this person thinking or imagining?",
"Which thoughts or opinions are subjective and which are objective?",
"What makes this person think the thought his thought is true?",
"Is there cognitive distortion in the speech?",
"What cognitive distortions are present in the speech? Please answer with a maximum of two cognitive distortions.",
]

def analyze_text_with_ollama(text):
    prompts[1] = "Given the following text, answer the questions in my following messages.\n\n" + text
    messages=[
            {"role": "system", "content": "You are a therpaist specializing in cognitive behavioural therapy."},
    ]
    for prompt in tqdm(prompts):
        messages.append({"role": "user", "content": prompt})
        response = ollama.chat(model="deepseek-r1:14b", messages=messages)
        messages.append({"role": "assistant", "content": response['message']['content']})
    return response['message']['content']

examples = [
    "If I can't get a good mark, then I'm talentless.",
    "My excitement will not allow me to perform on stage.",
    "My luck still allows me to hold this position.",
    "I feel that the event will be boring.",
    "My friend is short-sighted.",
    "My mistake at the meeting suggests that I do not know how to behave in society.",
    "Bonus does not mean that I do a good job.",
    "Despite the fact that I fulfilled the plan, my mistake in the report indicates that I am stupid.",
    "He or she thinks I'm not attractive / handsome.",
    "She / he talks to me rudely because I cannot explain my request.",
    "I have to get married before twenty-five because that's the way it is.",
    "It is you who made me feel bad.",
]

for example in tqdm(examples):
    print(analyze_text_with_ollama(example).split("</think>\n\n", 1)[1])

  0%|          | 0/12 [01:41<?, ?it/s]


KeyboardInterrupt: 